# The Guardian API

In the `05_web_scraping_beautiful_soup.ipynb` notebook, we saw examples on how BeautifulSoup can be used 
to parse messy HTML, to extract information, and to act as a rudimentary web crawler. 
We used The Guardian as an illustrative example about how this can be achieved. 
The reason for choosing The Guardian was because they provide a REST API to their servers. 
With the REST API it is possible to perform specific queries on their servers, and to receive 
current information from their servers according to their API guide (ie in JSON)

http://open-platform.theguardian.com/

In order to use their API, you will need to register for an API key. 
At the time of writing (Jan 24, 2022) this was an automated process that can be completed at 

https://bonobo.capi.gutools.co.uk/register/developer

On registration you will receive an API key which will look like: 303qwe2k-xxxx-xxxx-xxxx-eff86a248059

The API is documented here: 

http://open-platform.theguardian.com/documentation/

and Python bindings to their API are provided by The Guardian here

https://github.com/prabhath6/theguardian-api-python

and these can easily be integrated into a web-crawler based on API calls, rather than being based 
on HTML parsing, etc. 

We use four parameters in our queries here: 

1. `section`: the section of the newspaper that we are interested in querying. In this case we will look at 
the technology section 

2. `order-by`: We have specified that the newest items should be closer to the front of the query list 

3. `api-key`: In this notebook, the api-key is left as `test` (works here), but for *real* deployment of such a spider an API key obtained from Guardian should be specified. For the lab tasks, you should replace `test` API key with your personal API key. 

4. `page-size`: The number of results to return. 

In [1]:
from __future__ import print_function

import requests 
import json 

# Inspect all sections and search for technology-based sections

In [2]:
url = 'https://content.guardianapis.com/sections?api-key=3367cbf7-7e5a-406a-ab07-6a6c7e8f934d'
req = requests.get(url)
src = req.text 

In [3]:
json.loads(src)['response']['status']

'ok'

In [4]:
sections = json.loads(src)['response']

print(sections.keys())

dict_keys(['status', 'userTier', 'total', 'results'])


In [5]:
print(json.dumps(sections['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/about",
  "editions": [
    {
      "apiUrl": "https://content.guardianapis.com/about",
      "code": "default",
      "id": "about",
      "webTitle": "About",
      "webUrl": "https://www.theguardian.com/about"
    }
  ],
  "id": "about",
  "webTitle": "About",
  "webUrl": "https://www.theguardian.com/about"
}


In [6]:
for result in sections['results']: 
    if 'tech' in result['id'].lower(): 
        print(result['webTitle'], result['apiUrl'])

Technology https://content.guardianapis.com/technology


# Manual query on whole API

In [13]:
# Specify the arguments
args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': '3367cbf7-7e5a-406a-ab07-6a6c7e8f934d', 
    'page-size': '100',
    'q' : 'privacy%20AND%20data'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

In [14]:
print('Number of byes received:', len(src))

Number of byes received: 60446


The API returns JSON, so we parse this using the in-built JSON library. 
The API specifies that all data are returned within the `response` key, even under failure. 
Thereofre, I have immediately descended to the response field 

# Parsing the JSON

In [15]:
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']


# Verifying the status code

It is important to verify that the status message is `ok` before continuing - if it is not `ok` no 'real' data 
will have been received. 

In [16]:
assert response['status'] == 'ok'

# Listing the results 

The API standard states that the results will be found in the `results` field under the `response` field. 
Furthermore, the URLs will be found in the `webUrl` field, and the title will be found in the `webTitle` 
field. 

First let's look to see what a single result looks like in full, and then I will print a restricted 
set of parameters on the full set of results .

In [17]:
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/australia-news/2023/jan/25/dating-app-background-and-id-checks-being-considered-in-bid-to-fight-abuse",
  "id": "australia-news/2023/jan/25/dating-app-background-and-id-checks-being-considered-in-bid-to-fight-abuse",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2023-01-25T07:14:50Z",
  "webTitle": "Dating app background and ID checks being considered in bid to fight abuse",
  "webUrl": "https://www.theguardian.com/australia-news/2023/jan/25/dating-app-background-and-id-checks-being-considered-in-bid-to-fight-abuse"
}


In [18]:
for result in response['results']: 
    print(result['webUrl'][:70], result['webTitle'][:20])

https://www.theguardian.com/australia-news/2023/jan/25/dating-app-back Dating app backgroun
https://www.theguardian.com/technology/2023/jan/24/techscape-tiktok-ba TechScape: Is ‘banni
https://www.theguardian.com/technology/2023/jan/19/young-people-do-you Young people: do you
https://www.theguardian.com/technology/2023/jan/17/age-checks-trolls-a Age checks, trolls a
https://www.theguardian.com/technology/2023/jan/12/meta-voyager-labs-s Meta alleges surveil
https://www.theguardian.com/technology/2023/jan/09/amazon-employee-dea ‘Lack of respect’: o
https://www.theguardian.com/technology/2023/jan/09/afl-partner-cryptoc AFL partner Crypto.c
https://www.theguardian.com/technology/2023/jan/04/meta-dealt-blow-eu- Meta dealt blow by E
https://www.theguardian.com/technology/2022/dec/30/explainer-us-congre Why did the US just 
https://www.theguardian.com/technology/2022/dec/30/us-tiktok-ban-gover US bans China-based 
https://www.theguardian.com/technology/2022/dec/20/techscape-elon-musk TechScape